In [1]:
import os
cwd = os.getcwd()
print cwd

/home/ubuntu/insight-taxi-pulse/src/elasticsearch


In [2]:
# print(__doc__)
from kafka import KafkaConsumer
import json
from elasticsearch import Elasticsearch
import sys
import yaml
import signal
from datetime import datetime
import geohash

In [3]:
def yaml_loader(yaml_file):
	with open(yaml_file) as yml:
		config = yaml.load(yml)
	return config

In [4]:
def json_loader(json_file):
	with open(json_file) as jsn:
		config = json.load(jsn,object_hook=_tostring)
	return config

In [5]:
def signal_handler(signal,frame):
	print "--"
	print "--- elasticsearch consumer has been halted ---"
	print "--"
	sys.exit(0)

In [6]:
def _tostring(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _tostring(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {_tostring(key, ignore_dicts=True): _tostring(value, ignore_dicts=True) for key, value in data.iteritems()}
    # if it's anything else, return it in its original form
    return data

In [7]:
    import time
    t = time.time()
    elapsed = time.time() - t
    print elapsed

4.91142272949e-05


In [22]:

def consume_messages_new(topics):

    consumer = KafkaConsumer(bootstrap_servers=[port],
                             consumer_timeout_ms=5000 ,
                             value_deserializer=lambda m: json.loads(m.decode('ascii')) 
                             )
  
    consumer.subscribe(topics)
    count = 0
    print port

    for message in consumer:
        # check for stopping input
        signal.signal(signal.SIGINT , signal_handler)
        incoming_topic = message.topic
        incoming_message = message.value
        count = count + 1
        print "--------------"
        print incoming_topic
#         print message.value
        print incoming_message
        print "--------------"
        plat = incoming_message["pickup_latitude"];
        if plat is None:
            plat = 0
        plon = incoming_message["pickup_longitude"];
        if plon is None:
            plon = 0
        dlat = incoming_message["dropoff_latitude"];
        if dlat is None:
            dlat = 0
        dlon = incoming_message["dropoff_longitude"];
        if dlon is None:
            dlon = 0
        ts = incoming_message["timestamp"];
        if ts is None:
            ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        
        
        new_entry = {"pick_location": geohash.encode(plat, plon,precision=8),
                    "drop_location":  geohash.encode(dlat, dlon,precision=8),
                    "time_stamp": ts
                      }
        
        print new_entry
        es.index(index=incoming_topic, doc_type=incoming_topic[:-1], id=str(count), body =new_entry)



  

In [11]:
def consume_messages(topics):
    t = time.time()
    # stop iterations after 20 secs
    consumer = KafkaConsumer(bootstrap_servers=[port])
    consumer.subscribe(topics)
    count = 0
    print port

    for message in consumer:
        # check for stopping input
        signal.signal(signal.SIGINT , signal_handler)
        incoming_message = json.loads(message.value,object_hook=_tostring)
        incoming_topic = message.topic

        # round trip for consistent values
        # trip_dist = round(incoming_message["trip_distance"][str(0)],2)
        count = count + 1
        print "--------------"
        print incoming_topic
#         print message.value
        print incoming_message
        print "--------------"
        plat = incoming_message["pickup_latitude"];
        if plat is None:
            plat = 0
        plon = incoming_message["pickup_longitude"];
        if plon is None:
            plon = 0
        dlat = incoming_message["dropoff_latitude"];
        if dlat is None:
            dlat = 0
        dlon = incoming_message["dropoff_longitude"];
        if dlon is None:
            dlon = 0
        td = incoming_message["trip_distance"];
        if td is None:
            td = 0
        ts = incoming_message["timestamp"];
        if ts is None:
            ts = datetime.now().strftime("%Y-%m-%d'T'%H:%M:%S.%f")[:-3]
        
        
        new_entry = {"pick_location": {
                      "lat":  float(plat),
                      "lon":  float(plon)
                        },
                    "drop_location": {
                      "lat":  float(dlat),
                      "lon":  float(dlon)
                        },
                    "trip_distance": float(td),
                    "time_stamp": ts
                      }
        
        print new_entry
        es.index(index=incoming_topic, doc_type=incoming_topic[:-1], id=str(count), body =new_entry)


In [13]:
# load configuration parameters
config_path = 'config/stream_consumer_config.yml'
config = yaml_loader(config_path)

# initialize parameters
port = config['port']
topics = config['topics']
namenodeip = config['namenodeip']

# load mapping
mapping = json_loader('config/stream_elasticsearch_config.json')
print port
print topics
# print mapping
print namenodeip


ec2-34-193-153-112.compute-1.amazonaws.com:9092
['stream_users', 'stream_cars', 'cars', 'users']
34.193.153.112


In [14]:
es = Elasticsearch(http_auth=('elastic','changeme'))
print es

<Elasticsearch([{}])>


### Testing with example DB

this creates and populates the index at the same time.  
Kind of confusing if you are learning how to use elasticsearch.  
Not sure why this was an example.

In [15]:
from datetime import datetime
es = Elasticsearch(http_auth=('elastic','changeme'))
print 'testing index creation'
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", doc_type='tweet', id=1, body=doc)
print(res['created'])
res = es.get(index="test-index", doc_type='tweet', id=1)
print(res['_source'])

testing index creation
False
{u'text': u'Elasticsearch: cool. bonsai cool.', u'author': u'kimchy', u'timestamp': u'2017-02-05T16:07:22.274892'}


### Testing my mapping in ElasticSearch

In [16]:
map = {"users":{
    "mappings":{
      "user":{
        "_source": { "enabled": True },
        "properties": {
          "pick_location": {"type": "geo_point"},
          "drop_location": {"type": "geo_point"},
          "timestamp":     {"type": "date_hour_minute_second_millis",    "doc_values": True, "format": "yyyy-MM-dd'T'HH:mm:ss.SSS"}
            }
          }
      }
    }
    }
doc = {
    'text': 'heliotrope halcyon amarinthine.',
    'pick_location':  {'lat': 40, 'lon': -74},
    'drop_location': {'lat': 40, 'lon': -74},
    '@timestamp': datetime.now().strftime("%Y-%m-%d'T'%H:%M:%S.%f")[:-3],
}
es.indices.delete(index='testmaps',ignore=[400, 404])         
es.indices.create(index='testmaps', ignore=400, body=map)
res = es.index(index='testmaps', doc_type='users', id=1, body=doc)
print(res['created'])
res = es.get(index='testmaps', doc_type='users', id=1)
print(res['_source'])

True
{u'text': u'heliotrope halcyon amarinthine.', u'drop_location': {u'lat': 40, u'lon': -74}, u'@timestamp': u"2017-02-05'T'16:08:10.293", u'pick_location': {u'lat': 40, u'lon': -74}}


In [17]:
print topics[0]

# ignore 400 cause by IndexAlreadyExistsException when creating an index
res = es.indices.create(index=topics[0], ignore=400,body=mapping[topics[0]])

stream_users


In [18]:
### Remove the test indicies
es.indices.delete(index=topics[0], ignore=[400, 404])
es.indices.delete(index='my-index', ignore=[400, 404])
es.indices.delete(index='test-index', ignore=[400, 404])
es.indices.delete(index='twitter', ignore=[400, 404])
es.indices.delete(index='cars', ignore=[400, 404])
es.indices.delete(index='users', ignore=[400, 404])


{u'acknowledged': True}

### Testing for all mappings

In [19]:
print "subscribing elastic search to the following topics: "+str(topics)

# create index
for index in topics:
    print index
    es.indices.delete(index=index,ignore=[400, 404])  
    es.indices.create(index=index, ignore=400, body=mapping[index])  
#     doc = {
#     "pickup_latitude": "43.001",
#     'pickup_longitude': '72.001',
#     'dropoff_latitude': '43.001',
#     'dropoff_longitude': '72.001',
#     'time_stamp': datetime.now(),
#     }
#     res = es.index(index=index, doc_type='test-type', id=1, body=doc)

subscribing elastic search to the following topics: ['stream_users', 'stream_cars', 'cars', 'users']
stream_users
stream_cars
cars
users


### listing out all existing Indices

indicies will not be found (do not completely exist)  
until they have been populated with at least one item

In [20]:
for index in es.indices.get('*'):
  print index

.monitoring-data-2
timetest
users
testmaps
cars
attractions
my_index
.monitoring-es-2-2017.02.03
.monitoring-es-2-2017.02.01
.kibana
.monitoring-es-2-2017.02.04
.monitoring-es-2-2017.02.05
.monitoring-kibana-2-2017.02.04
.monitoring-kibana-2-2017.02.05
.monitoring-kibana-2-2017.02.03
.monitoring-es-2-2017.01.30
.monitoring-es-2-2017.01.31


## (should stop after 5 secs) <span style="color:red">  - get kafka running producer first! </span>

In [23]:
# consume messages from Kafka
consume_messages_new(topics)


ec2-34-193-153-112.compute-1.amazonaws.com:9092
--------------
users
{u'trip_distance': 1.34, u'tolls_amount': 0, u'pickup_datetime': u'2013-05-01 00:04:00', u'tip_amount': 0, u'total_amount': 8, u'mta_tax': 0.5, u'vendor_id': u'VTS', u'fare_amount': 7, u'pickup_longitude': -73.982287, u'dropoff_longitude': -73.98621, u'passenger_count': 1, u'surcharge': 0.5, u'dropoff_datetime': u'2013-05-01 00:12:00', u'timestamp': u'2017-02-05 16:09:30.744', u'store_and_fwd_flag': None, u'rate_code': 1, u'pickup_latitude': 40.772815, u'dropoff_latitude': 40.758742, u'payment_type': u'CSH'}
--------------
{'time_stamp': u'2017-02-05 16:09:30.744', 'drop_location': 'dr5ru7uu', 'pick_location': 'dr5ruqrw'}
--------------
users
{u'trip_distance': 2.6, u'tolls_amount': 0, u'pickup_datetime': u'2013-05-01 00:03:00', u'tip_amount': 2, u'total_amount': 12.5, u'mta_tax': 0.5, u'vendor_id': u'VTS', u'fare_amount': 9.5, u'pickup_longitude': -73.96300999999998, u'dropoff_longitude': -73.99187499999998, u'passen

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [19]:
es.search(index='cars', filter_path=['hits.hits._id', 'hits.hits._drop_location'])


{}

In [19]:
testq = {
    "query" : { "match_all" : { } }
}

# body={"query": {"match_all": {}}}
res = es.search(index="cars", doc_type="car", body=testq)
print("%d documents found" % res['hits']['total'])
for doc in res['hits']['hits']:
    print("%s) %s %s" % (doc['_id'], doc['_source']['drop_location'], doc['_source']['timestamp']))



1 documents found
2) {u'lat': 40.758742, u'lon': -73.98621} 2017-02-03 18:10:13.458


In [75]:
testq = {
    "query" : { "match_all" : { } }
}

testq2 = {
  "filtered" : {
    "query" : { "match_all" : { } },
        "filter" : {
            "match_all" : { }
            }
        }
    }


search_body = {
    "constant_score" : {
        "filter" : {
            "exists" : { "field" : "drop_location" }
        }
    }
}

result = es.search(index="cars", doc_type="car", body=testq2)
for hit in result["hits"]["hits"]:
    print hit["_source"]["drop_location"]

    
# res = es.search(index="cars", doc_type="car", body=testq)
# print("%d documents found" % res['hits']['total'])
# for doc in res['hits']['hits']:
#     print("%s) %s" % (doc['_id'], doc['_source']['drop_location']))

RequestError: TransportError(400, u'parsing_exception', u'Unknown key for a START_OBJECT in [filtered].')

In [70]:
qt =  { 
    "bool": { 
      "must": [
        { "match": { "title":   "Search"        }}, 
        { "match": { "content": "Elasticsearch" }}  
      ],
      "filter": [ 
        { "term":  { "status": "published" }}, 
        { "range": { "publish_date": { "gte": "2015-01-01" }}} 
      ]
    }
  }

In [58]:
for index in es.indices.get('*'):
  print index

test-index
.monitoring-data-2
users
cars
.monitoring-es-2-2017.02.03
.monitoring-es-2-2017.02.01
.monitoring-es-2-2017.01.30
.monitoring-es-2-2017.01.31
